In [17]:
import torch
import torchvision
import torchvision.transforms as tfs




In [24]:
transform = tfs.Compose([
    tfs.RandomCrop(32, padding=4),
    tfs.RandomHorizontalFlip(),
    tfs.ToTensor(),
    tfs.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', download= True, train=True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=6, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data',download = True,  train=False, transform= transform)
testloader = torch.utils.data.DataLoader(testset,batch_size=6, shuffle= False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat','deer',
           'dog', 'frog', 'horse','ship', 'truck')



Files already downloaded and verified
Files already downloaded and verified


In [26]:
print("Dataset Details:")
print("Number of samples:", len(trainset))
print("Number of classes:", len(trainset.classes))
print("Classes:", trainset.classes)
print("Class to index mapping:", trainset.class_to_idx)
print("Sample shape:", trainset[0][0].shape)



Dataset Details:
Number of samples: 50000
Number of classes: 10
Classes: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
Class to index mapping: {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Sample shape: torch.Size([3, 32, 32])


In [19]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(128 * 4 * 4, 10)  # 10 classes in CIFAR-10

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = self.fc(x)
        return x
    
net = Net()



In [20]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [21]:
for epoch in range(10):
    running_loss = 0
    for i,data in enumerate(trainloader, 0):
        inputs, labels =data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        
print('Finished Training')
        

        


[1,  2000] loss: 2.012
[1,  4000] loss: 1.687
[1,  6000] loss: 1.559
[1,  8000] loss: 1.447
[2,  2000] loss: 1.386
[2,  4000] loss: 1.308
[2,  6000] loss: 1.248
[2,  8000] loss: 1.189
[3,  2000] loss: 1.137
[3,  4000] loss: 1.095
[3,  6000] loss: 1.043
[3,  8000] loss: 1.018
[4,  2000] loss: 0.968
[4,  4000] loss: 0.982
[4,  6000] loss: 0.934
[4,  8000] loss: 0.924
[5,  2000] loss: 0.893
[5,  4000] loss: 0.882
[5,  6000] loss: 0.865
[5,  8000] loss: 0.863
[6,  2000] loss: 0.825
[6,  4000] loss: 0.822
[6,  6000] loss: 0.804
[6,  8000] loss: 0.808
[7,  2000] loss: 0.790
[7,  4000] loss: 0.776
[7,  6000] loss: 0.766
[7,  8000] loss: 0.762
[8,  2000] loss: 0.745
[8,  4000] loss: 0.752
[8,  6000] loss: 0.721
[8,  8000] loss: 0.735
[9,  2000] loss: 0.703
[9,  4000] loss: 0.720
[9,  6000] loss: 0.698
[9,  8000] loss: 0.712
[10,  2000] loss: 0.683
[10,  4000] loss: 0.702
[10,  6000] loss: 0.665
[10,  8000] loss: 0.684
Finished Training


In [22]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 75 %


In [23]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
#confusion matrix for the predictions
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

#shape of each image in cifar10


